In [3]:
#import libraries
import requests
import pandas as pd
from datetime import datetime
import os
import html

In [53]:
#Add trafic2023_ratio or raw somewhere?
##Add holidays

In [5]:
#download dataset
df_lines = pd.read_csv("./Online_Data/referentiel-des-lignes.csv", sep=';')
stops_data = pd.read_csv("./Online_Data/arrets.csv", sep=';')

trafic2023_ratio = pd.read_csv("./Online_Data/validations-1er-semestre.csv", sep=';')
trafic2023_raw = pd.read_csv("./Online_Data/validations-reseau.csv", sep=";")

hourly_weather = pd.read_csv("./Online_Data/Weather/hourly_weather.csv")
minutely_15_weather = pd.read_csv("./Online_Data/Weather/minutely_15_weather.csv")

delays_metro = pd.read_csv("./Collected_Data/metro_delays.csv")
delays_rer = pd.read_csv("./Collected_Data/rail_delays.csv")
onTime_metro = pd.read_csv("./Collected_Data/metro_onTime.csv")
onTime_rer = pd.read_csv("./Collected_Data/rail_onTime.csv")

metro_incident = pd.read_csv("./Collected_Data/metro_line_reports.csv")
rer_incident = pd.read_csv("./Collected_Data/rer_line_reports.csv")
#gas = pd.read_csv("./Online_Data/prix-des-carburants-en-france-flux-instantane-v2.csv", sep=";") #needs cleaning/filtering

In [6]:
#create filter
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
name_filter = ["CH.D.G.ETOILE", "CHATELET", "SAINT-LAZARE","ST-GERM.D.PRES", "BLANCHE","AVENUE DU PRESIDENT KENNEDY","BUNO GIRONVILLE","MASSY PALAISEAU"]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [7]:
#clean dataset
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

def cleaning_message(df):
    df['ref'] = df['ref'].str.replace('stop_point:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('line:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('stop_area:IDFM:', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<p>', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<br>', '', regex=False) 
    df = df[df["channel_name"].isin(["moteur"])]

cleaning_message(metro_incident)
cleaning_message(rer_incident)

metro_incident['message_text'] = metro_incident['message_text'].str.split('</p>').str[0]
metro_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
metro_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

rer_incident['message_text'] = rer_incident['message_text'].str.replace('</p>', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].apply(html.unescape)
rer_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
rer_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

delays_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
delays_metro['line_ref'] = delays_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')
onTime_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
onTime_metro['line_ref'] = onTime_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')

onTime_metro['stop_reference'] = pd.to_numeric(onTime_metro['stop_reference'], errors='coerce')
onTime_metro['stop_reference'] = onTime_metro['stop_reference'].fillna(0).astype('int64')
onTime_rer['stop_reference'] = pd.to_numeric(onTime_rer['stop_reference'], errors='coerce')
onTime_rer['stop_reference'] = onTime_rer['stop_reference'].fillna(0).astype('int64')

mapping = dict(zip(name_filter, stops_filter))
trafic2023_ratio['LIBELLE_ARRET_REA'] = trafic2023_ratio['LIBELLE_ARRET'].replace(mapping)
trafic2023_ratio.drop(["lda"],axis=1, inplace=True)

trafic2023_raw['LIBELLE_ARRET_REA'] = trafic2023_raw['LIBELLE_ARRET'].replace(mapping)
trafic2023_raw.drop(["lda"],axis=1, inplace=True)

In [8]:
#filter dataset
stops_data = stops_data[stops_data['ArRId'].isin(stops_filter)]

metro_incident = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

rer_incident = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

trafic2023_ratio = trafic2023_ratio[trafic2023_ratio['LIBELLE_ARRET_REA'].isin(stops_filter)]

trafic2023_raw = trafic2023_raw[trafic2023_raw['LIBELLE_ARRET_REA'].isin(stops_filter)]

In [9]:
#Merge datasets
merged_metro = pd.concat([delays_metro, onTime_metro], ignore_index=True)
merged_metro = pd.merge(merged_metro, stops_data, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'], axis=1, inplace=True) 
merged_metro = merged_metro.sort_values(by='real_arrival')

merged_rer = pd.concat([delays_rer, onTime_rer], ignore_index=True)
merged_rer = pd.merge(merged_rer, stops_data, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'],  axis=1, inplace=True)
merged_rer = merged_rer.sort_values(by='scheduled_arrival')

def date_format(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('Z', ''), errors='coerce')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['real_arrival'] = pd.to_datetime(df['real_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_arrival'] = pd.to_datetime(df['real_arrival'])
    df['real_departure'] = pd.to_datetime(df['real_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_departure'] = pd.to_datetime(df['real_departure'])


def format_rer(df):
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival'])
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure'])
    df.loc[(df['arrival_difference'] >= 5) | (df['departure_difference'] >= 5), 'departure_status'] = 'delayed'
    df['nearest_datetime'] = df['scheduled_arrival'].combine_first(df['scheduled_departure'])

date_format(merged_metro)
date_format(merged_rer)
format_rer(merged_rer)

In [11]:
#Clean Gas Price data
def clean_gas_data(file_path, fuel_type):
    # Load CSV file
    df = pd.read_csv(file_path, skiprows=3, sep=';')  # Skip the first 3 metadata rows
    df.rename(columns={df.columns[1]: f'{fuel_type}'}, inplace=True)
    df = df.iloc[:, :-1]
    return df
    
gas_95_df = clean_gas_data("./Online_Data/Gas/octane_95.csv", "95")
gas_98_df = clean_gas_data("./Online_Data/Gas/octane_98.csv", "98")
gas_e10_df = clean_gas_data("./Online_Data/Gas/95-E10.csv", "E10")
gazole_df = clean_gas_data("./Online_Data/Gas/gazole.csv", "gazole")

merged_metro = merged_metro.sort_values(by='real_arrival')
merged_rer = merged_rer.sort_values(by='nearest_datetime')

merged_rer['Période'] = pd.to_datetime(merged_rer['nearest_datetime']).dt.strftime('%Y-%m')
merged_rer = merged_rer.merge(gas_95_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_98_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_e10_df, on='Période', how='left')
merged_rer = merged_rer.merge(gazole_df, on='Période', how='left')

In [12]:
def days_and_hols(df):
    df['day'] = df['timestamp'].dt.date
    df['day_of_week'] = df['timestamp'].dt.day_name()
    df['is_weekend']= df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)

days_and_hols(merged_rer)
days_and_hols(merged_metro)


merged_metro.drop(columns=['timestamp'], inplace=True)
merged_rer.drop(columns=['Période', 'timestamp'], inplace=True)
merged_rer = merged_rer[~merged_rer.duplicated(subset=merged_rer.columns.difference(['delayed_status']).tolist(), keep='last')]

In [14]:
#hourly_weather['date'] = pd.to_datetime(hourly_weather['date'])
minutely_15_weather['date'] = pd.to_datetime(minutely_15_weather['date'])

merged_metro = pd.merge_asof(merged_metro, 
                   minutely_15_weather, 
                   left_on='real_arrival', 
                   right_on='date', 
                   direction='nearest')

merged_rer = pd.merge_asof(merged_rer, 
                   minutely_15_weather, 
                   left_on='nearest_datetime', 
                   right_on='date', 
                   direction='nearest')

In [48]:
#split by stop
CDG = merged_metro[merged_metro['stop_name'].isin(['Charles de Gaulle-Etoile'])]
CDG = CDG.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')

SGP = merged_metro[merged_metro['stop_name'].isin(['Saint-Germain des Prés'])]
SGP = SGP.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')

BL = merged_metro[merged_metro['stop_name'].isin(['Blanche'])]
BL = BL.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')

SL = merged_metro[merged_metro['stop_name'].isin(['Saint-Lazare'])]
SL = SL.drop_duplicates(subset=['destination_name', 'real_arrival'], keep='first')

APK = merged_rer[merged_rer['stop_name'].isin(['Avenue du Président Kennedy'])]
APK = APK.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')

CLH = merged_rer[merged_rer['stop_name'].isin(['Châtelet - Les Halles'])]
CLH = CLH.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')

GBG = merged_rer[merged_rer['stop_name'].isin(['Gare de Buno Gironville'])]
GBG = GBG.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')

MP = merged_rer[merged_rer['stop_name'].isin(['Massy - Palaiseau'])]
MP = MP.drop_duplicates(subset=['destination_name', 'scheduled_arrival'], keep='first')

In [50]:
CDG.to_csv('CDG.csv', index=False)
SGP.to_csv('SGP.csv', index=False)
BL.to_csv('BL.csv', index=False)
SL.to_csv('SL.csv', index=False)
APK.to_csv('APK.csv', index=False)
CLH.to_csv('CLH.csv', index=False)
GBG.to_csv('GBG.csv', index=False)
MP.to_csv('MP.csv', index=False)